In [1]:
# 挖掘数据中的关联规则

In [1]:
from efficient_apriori import apriori

import os
import warnings
warnings.filterwarnings('ignore')
path = os.path.expanduser('~/data/')

In [49]:
# 设置数据集
transactions = [('牛奶','面包','尿布'),
		('可乐','面包', '尿布', '啤酒'),
		('牛奶','尿布', '啤酒', '鸡蛋'),
		('面包', '牛奶', '尿布', '啤酒'),
		('面包', '牛奶', '尿布', '可乐')]

In [50]:
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(transactions, min_support=0.5,  min_confidence=1)
print("频繁项集：", itemsets)
print("关联规则：", rules)

频繁项集： {1: {('尿布',): 5, ('面包',): 4, ('牛奶',): 4, ('啤酒',): 3}, 2: {('尿布', '牛奶'): 4, ('尿布', '面包'): 4, ('牛奶', '面包'): 3, ('啤酒', '尿布'): 3}, 3: {('尿布', '牛奶', '面包'): 3}}
关联规则： [{牛奶} -> {尿布}, {面包} -> {尿布}, {啤酒} -> {尿布}, {牛奶, 面包} -> {尿布}]


In [2]:
## BreadBasket

In [3]:
import pandas as pd
import time

In [4]:
# 数据加载
data = pd.read_csv(path +'/BI/BreadBasket_DMS.csv')
# 统一小写
data['Item'] = data['Item'].str.lower()
# 去掉none项
data = data.drop(data[data.Item == 'none'].index)

In [9]:
# 采用efficient_apriori工具包
def rule1():
	from efficient_apriori import apriori
	start = time.time()
	# 得到一维数组orders_series，并且将Transaction作为index, value为Item取值
	orders_series = data.set_index('Transaction')['Item']
	# 将数据集进行格式转换
	transactions = []
	temp_index = 0
	for i, v in orders_series.items():
		if i != temp_index:
			temp_set = set()
			temp_index = i
			temp_set.add(v)
			transactions.append(temp_set)
		else:
			temp_set.add(v)
	
	# 挖掘频繁项集和频繁规则
	itemsets, rules = apriori(transactions, min_support=0.02,  min_confidence=0.5)
	print('频繁项集：', itemsets)
	print('关联规则：', rules)
	end = time.time()
	print("用时：", end-start)

In [10]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [13]:
# 采用mlxtend.frequent_patterns工具包
def rule2():
	from mlxtend.frequent_patterns import apriori
	from mlxtend.frequent_patterns import association_rules
	pd.options.display.max_columns=100
	start = time.time()
	hot_encoded_df=data.groupby(['Transaction','Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')
	hot_encoded_df = hot_encoded_df.applymap(encode_units)
	frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
	rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
	print("频繁项集：", frequent_itemsets)
	print("关联规则：", rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5) ])
	#print(rules['confidence'])
	end = time.time()
	print("用时：", end-start)

In [14]:
rule1()
print('-'*100)
rule2()

频繁项集： {1: {('scandinavian',): 275, ('cookies',): 515, ('hot chocolate',): 552, ('muffin',): 364, ('coffee',): 4528, ('pastry',): 815, ('bread',): 3096, ('medialuna',): 585, ('tea',): 1350, ('farm house',): 371, ('juice',): 365, ('soup',): 326, ('cake',): 983, ('sandwich',): 680, ('alfajores',): 344, ('brownie',): 379, ('truffles',): 192, ('toast',): 318, ('scone',): 327}, 2: {('bread', 'coffee'): 852, ('bread', 'pastry'): 276, ('coffee', 'pastry'): 450, ('coffee', 'medialuna'): 333, ('coffee', 'tea'): 472, ('bread', 'tea'): 266, ('coffee', 'juice'): 195, ('coffee', 'hot chocolate'): 280, ('coffee', 'cookies'): 267, ('cake', 'coffee'): 518, ('cake', 'tea'): 225, ('bread', 'cake'): 221, ('coffee', 'sandwich'): 362, ('coffee', 'toast'): 224}}
关联规则： [{pastry} -> {coffee}, {medialuna} -> {coffee}, {juice} -> {coffee}, {hot chocolate} -> {coffee}, {cookies} -> {coffee}, {cake} -> {coffee}, {sandwich} -> {coffee}, {toast} -> {coffee}]
用时： 0.12102985382080078
----------------------------------

In [15]:
## MovieActors

In [16]:
### 分析MovieLens 电影分类中的频繁项集和关联规则

In [17]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [5]:
# 数据加载
movies = pd.read_csv(path+'/BI/movie_actors.csv')
#print(movies.head())

In [19]:
# 将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
movies_hot_encoded = movies.drop('actors',1).join(movies.actors.str.get_dummies('/'))
pd.options.display.max_columns=100
print(movies_hot_encoded.head())

            title  GangZhao  LukeZhiGangLiu  PengZhenZhong  YuanFang  一纳  丁嘉丽  \
0      囧妈‎ (2020)         0               0              0         0   0    0   
1  我和我的祖国‎ (2019)         0               0              0         0   0    0   
2   我不是药神‎ (2018)         0               0              0         0   0    0   
3  疯狂的外星人‎ (2019)         0               0              0         0   0    0   
4   疯狂的石头‎ (2006)         0               0              0         0   0    0   

   丁志城  丁志诚  丁黑  万弘杰  严敏  严晓频  中孝介  丹尼尔·海尼  乔任梁  九孔  于和伟  于波  于荣光  于谦  仁龙  \
0    0    0   0    0   0    0    0       0    0   0    0   0    0   0   0   
1    0    0   0    0   0    0    0       0    0   0    0   0    0   0   0   
2    0    0   0    0   0    0    0       0    0   0    0   0    0   0   0   
3    0    0   0    0   0    0    0       0    0   0    1   0    0   0   0   
4    0    0   0    0   0    0    0       0    0   0    0   0    0   0   0   

   付连智  任达华  任静  任鹏远  伊一  伊春德  伊能静  伊莎贝尔·于佩尔  优恵  

/Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [20]:
# 将movieId, title设置为index
movies_hot_encoded.set_index(['title'],inplace=True)
#print(movies_hot_encoded.head())

In [21]:
# 挖掘频繁项集，最小支持度为0.02
itemsets = apriori(movies_hot_encoded,use_colnames=True, min_support=0.05)
# 按照支持度从大到小进行时候粗
itemsets = itemsets.sort_values(by="support" , ascending=False) 
print('-'*20, '频繁项集', '-'*20)
print(itemsets)

-------------------- 频繁项集 --------------------
     support      itemsets
2   0.768421          (徐峥)
7   0.157895          (黄渤)
11  0.094737      (黄渤, 徐峥)
1   0.073684          (宁浩)
0   0.063158         (于和伟)
6   0.063158          (陶虹)
8   0.063158      (宁浩, 徐峥)
9   0.063158      (黄渤, 宁浩)
13  0.063158  (宁浩, 黄渤, 徐峥)
3   0.052632         (王宝强)
4   0.052632          (王迅)
5   0.052632         (陈凯歌)
10  0.052632      (徐峥, 陶虹)
12  0.052632      (黄渤, 王迅)


In [22]:
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=2)

In [23]:
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False) 
#rules.to_csv('./rules.csv')
print('-'*20, '关联规则', '-'*20)
print(rules)

-------------------- 关联规则 --------------------
  antecedents consequents  antecedent support  consequent support   support  \
3    (黄渤, 徐峥)        (宁浩)            0.094737            0.073684  0.063158   
4        (宁浩)    (黄渤, 徐峥)            0.073684            0.094737  0.063158   
2    (徐峥, 宁浩)        (黄渤)            0.063158            0.157895  0.063158   
7        (王迅)        (黄渤)            0.052632            0.157895  0.052632   
5        (黄渤)    (徐峥, 宁浩)            0.157895            0.063158  0.063158   
6        (黄渤)        (王迅)            0.157895            0.052632  0.052632   
0        (黄渤)        (宁浩)            0.157895            0.073684  0.063158   
1        (宁浩)        (黄渤)            0.073684            0.157895  0.063158   

   confidence      lift  leverage  conviction  
3    0.666667  9.047619  0.056177    2.778947  
4    0.857143  9.047619  0.056177    6.336842  
2    1.000000  6.333333  0.053186         inf  
7    1.000000  6.333333  0.044321         inf  
5

In [24]:
### 分析MovieLens 电影分类中的频繁项集和关联规则

In [25]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [6]:
# 数据加载
movies = pd.read_csv(path+'/BI/movies.csv')
#print(movies.head())


In [27]:
# 将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
print(movies['genres'])
movies_hot_encoded = movies.drop('genres',1).join(movies.genres.str.get_dummies(sep='|'))
print(movies_hot_encoded)

pd.options.display.max_columns=100
print(movies_hot_encoded.head())

0        Adventure|Animation|Children|Comedy|Fantasy
1                         Adventure|Children|Fantasy
2                                     Comedy|Romance
3                               Comedy|Drama|Romance
4                                             Comedy
                            ...                     
27273                                         Comedy
27274                                         Comedy
27275                                      Adventure
27276                             (no genres listed)
27277                       Adventure|Fantasy|Horror
Name: genres, Length: 27278, dtype: object
       movieId                               title  (no genres listed)  \
0            1                    Toy Story (1995)                   0   
1            2                      Jumanji (1995)                   0   
2            3             Grumpier Old Men (1995)                   0   
3            4            Waiting to Exhale (1995)                   0   
4   

/Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
# 将movieId, title设置为index
movies_hot_encoded.set_index(['movieId','title'],inplace=True)
#print(movies_hot_encoded.head())
# 挖掘频繁项集，最小支持度为0.02
itemsets = apriori(movies_hot_encoded,use_colnames=True, min_support=0.02)
# 按照支持度从大到小进行时候粗
itemsets = itemsets.sort_values(by="support" , ascending=False) 
print('-'*20, '频繁项集', '-'*20)
print(itemsets)

-------------------- 频繁项集 --------------------
     support                  itemsets
7   0.489185                   (Drama)
4   0.306987                  (Comedy)
14  0.153164                (Thriller)
12  0.151294                 (Romance)
0   0.129042                  (Action)
5   0.107743                   (Crime)
9   0.095718                  (Horror)
31  0.094325          (Romance, Drama)
26  0.093335           (Comedy, Drama)
6   0.090586             (Documentary)
1   0.085380               (Adventure)
27  0.069470         (Romance, Comedy)
32  0.068480         (Drama, Thriller)
13  0.063898                  (Sci-Fi)
28  0.062761            (Crime, Drama)
11  0.055503                 (Mystery)
8   0.051763                 (Fantasy)
29  0.045165         (Crime, Thriller)
20  0.044101           (Action, Drama)
15  0.043772                     (War)
3   0.041755                (Children)
22  0.040655        (Action, Thriller)
34  0.039336        (Thriller, Horror)
10  0.037979     

In [29]:
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=2)

In [30]:
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False) 
#rules.to_csv('./rules.csv')
print('-'*20, '关联规则', '-'*20)
print(rules)

-------------------- 关联规则 --------------------
          antecedents        consequents  antecedent support  \
9           (Mystery)         (Thriller)            0.055503   
8          (Thriller)          (Mystery)            0.153164   
14            (Crime)  (Drama, Thriller)            0.107743   
13  (Drama, Thriller)            (Crime)            0.068480   
7         (Adventure)           (Action)            0.085380   
6            (Action)        (Adventure)            0.129042   
16           (Action)           (Sci-Fi)            0.129042   
17           (Sci-Fi)           (Action)            0.063898   
1          (Thriller)            (Crime)            0.153164   
0             (Crime)         (Thriller)            0.107743   
5            (Horror)         (Thriller)            0.095718   
4          (Thriller)           (Horror)            0.153164   
12     (Crime, Drama)         (Thriller)            0.062761   
15         (Thriller)     (Crime, Drama)            0.153

In [31]:
## regression

In [32]:
"""
	使用sklearn自带的糖尿病数据集，进行回归分析
	Diabetes：包含442个患者的10个生理特征（年龄，性别、体重、血压）和一年以后疾病级数指标
"""
from sklearn import datasets
from sklearn import linear_model
from sklearn.model_selection import train_test_split  
from sklearn.metrics import mean_squared_error

In [33]:
# 加载数据
diabetes = datasets.load_diabetes()
data = diabetes.data
#import numpy as np
#print(np.max(data))
#data.to_csv('diabetes.csv')

In [34]:
# 数据探索
print(data.shape)
print(data[0])

(442, 10)
[ 0.03807591  0.05068012  0.06169621  0.02187235 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990842 -0.01764613]


In [35]:
# 训练集 70%，测试集30%
train_x, test_x, train_y, test_y = train_test_split(diabetes.data, diabetes.target, test_size=0.3, random_state=14)
print(len(train_x))

309


In [36]:
#回归训练及预测
clf = linear_model.LinearRegression()
clf.fit(train_x, train_y)

print(clf.coef_)

[  32.03000032 -228.38626681  492.80665731  313.61844116 -991.31389923
  551.99413533  190.16297006  278.51146815  781.03825662   72.08348977]


In [37]:
#print(train_x.shape)
#print(clf.score(test_x, test_y))
pred_y = clf.predict(test_x)
print(mean_squared_error(test_y, pred_y))
r_sq = clf.score(train_x, train_y) #确定系数
print('r_sq:', r_sq)

3180.3670319563726
r_sq: 0.5194074106259234


In [38]:
### Pearson

In [39]:
import pandas as pd

def compute(x):
    return 2*x*x+1
x=[i for i in range(100)]
y=[compute(i) for i in x]
data = pd.DataFrame({'x':x,'y':y})
# 查看pearson系数
print(data.corr())
print(data.corr(method='spearman'))
print(data.corr(method='kendall'))

          x         y
x  1.000000  0.967644
y  0.967644  1.000000
     x    y
x  1.0  1.0
y  1.0  1.0
     x    y
x  1.0  1.0
y  1.0  1.0


In [40]:
### 回归分析
import random
from sklearn import linear_model

In [41]:
reg = linear_model.LinearRegression()

In [42]:
def generate(x):
	y = 2*x+10+random.random()
	return y

In [44]:
train_x = []
train_y = []
for x in range(1000):
	train_x.append([x])
	y = generate(x)
	train_y.append([y])

In [45]:
reg.fit (train_x, train_y)
# coef_ 保存线性模型的系数w
print(reg.coef_)
print(reg.intercept_)

[[1.99995684]]
[10.52414906]


In [51]:
## Market Basket

In [7]:
from efficient_apriori import apriori
trade_list=[]
with open(path + "/BI/Market_Basket_Optimisation.csv") as ff:
	for line in ff:
		line=line.strip()
		trade=line.split(',')
		trade_list.append(trade)

In [53]:
#使用Apriori算法
itemsets, rules = apriori(trade_list, min_support=0.05,  min_confidence=0.3)
print("频繁项集：", itemsets)
print("关联规则：", rules)

频繁项集： {1: {('frozen smoothie',): 475, ('low fat yogurt',): 574, ('shrimp',): 536, ('olive oil',): 494, ('mineral water',): 1788, ('green tea',): 991, ('eggs',): 1348, ('burgers',): 654, ('turkey',): 469, ('whole wheat rice',): 439, ('milk',): 972, ('french fries',): 1282, ('soup',): 379, ('frozen vegetables',): 715, ('spaghetti',): 1306, ('cookies',): 603, ('cooking oil',): 383, ('chicken',): 450, ('chocolate',): 1229, ('tomatoes',): 513, ('pancakes',): 713, ('grated cheese',): 393, ('escalope',): 595, ('ground beef',): 737, ('cake',): 608}, 2: {('eggs', 'mineral water'): 382, ('mineral water', 'spaghetti'): 448, ('chocolate', 'mineral water'): 395}}
关联规则： [{spaghetti} -> {mineral water}, {chocolate} -> {mineral water}]
